This notebook processes a SHG maximum-intensity-procession, prepared it for CurveAlign, and then processes the results.

In [29]:
from pathlib import Path
import multiscale.toolkits.curve_align as ca
import pandas as pd
import numpy as np
import multiscale.itk.metadata as meta
import multiscale.tiling as til
import SimpleITK as sitk

Format the image into tiles

In [2]:
# Specify the input directory where a 2D SHG file is.  This function processes ALL files in a folder
input_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\SHG CA')
output_dir = Path(input_dir, 'Tiles')
output_suffix = 'SHG'
batch_size = 5

In [ ]:
ca.create_batches_for_chtc(input_dir, output_dir, output_suffix, batch_size=batch_size)

The next step is to process the entirety of the "Batches" folder using the CHTC version of CurveAlign.  The job file is in the Batches folder.  See Yuming Liu for instructions and help on how to do so.

Once the CurveAlign analysis has finished, you now extract the statistical results from the tar files and then delete the tars, as they each contain images which consume a lot of hard drive space. 

In [34]:
tar_dir = Path(r'F:\CA Output')
statistics_dir = Path(input_dir, 'Statistics')
ca.bulk_extract_tar(tar_dir, statistics_dir)

Now you process the tile and ROI results into a single dataframe for easy analysis and image reconstruction

In [35]:
ca.scrape_results(statistics_dir, '' , 'SHG')

Scraping results from F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\SHG CA\Statistics\images\CA_Out
Done
True
Scraping results from F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\SHG CA\Statistics\images\CA_ROI\Batch\ROI_post_analysis


In [24]:
results_file = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\SHG CA\Statistics\ROI', 'Curve_Align_results_ROIs_SHG.csv')
df_results = pd.read_csv(results_file, header=[0], index_col=[0, 1, 2, 3])

In [14]:
def get_num_rois(path_to_image, tile_size=np.array([512, 512])):
        size_full = meta.get_image_size_from_path(path_to_image)
        num_tiles, offset_tile = til.calculate_number_of_tiles(size_full, tile_size)
        size_roi = np.array(num_tiles) * 8
        
        return size_roi

In [17]:
base_image_path = Path(input_dir, 'Mouse-210_SHG.tif')
#
dimensions = get_num_rois(base_image_path)


In [26]:
df_results['Alignment']

Sample     Modality  Tile    ROI    
Mouse-210  SHG       0x-13y  ROI5x4y    0.12
                     0x-15y  ROI3x6y    0.78
                             ROI3x7y    0.28
                             ROI4x7y    1.00
                             ROI5x5y    1.00
                             ROI5x6y    0.80
                             ROI6x6y    1.00
                     0x-16y  ROI7x7y    0.99
                     0x-17y  ROI0x7y    0.97
                             ROI1x7y    0.83
                             ROI2x7y    0.90
                             ROI3x7y    0.91
                             ROI4x6y    0.97
                             ROI4x7y    0.83
                             ROI5x6y    0.65
                             ROI5x7y    0.70
                             ROI6x6y    0.64
                             ROI6x7y    0.46
                             ROI7x6y    1.00
                             ROI7x7y    0.16
                     0x-18y  ROI0x6y    0.55
                  

In [27]:
alignment_array = til.roi_values_to_sitk_image_array(df_results['Alignment'], dimensions, 'SHG')

c:\users\mpinkert\documents\github\multiscale\multiscale\tiling.py:268: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  value = pd_series.get_value(i, col_label)


In [28]:
def write_image(image_array, path_image):
        image = sitk.Cast(sitk.GetImageFromArray(np.transpose(image_array)), sitk.sitkFloat32)
        sitk.WriteImage(image, str(path_image))

In [31]:
align_path = Path(input_dir, 'Mouse-210_Alignment.tif')
write_image(alignment_array, align_path)


In [32]:
orientation_array = til.roi_values_to_sitk_image_array(df_results['Orientation'], dimensions, 'SHG')

c:\users\mpinkert\documents\github\multiscale\multiscale\tiling.py:268: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  value = pd_series.get_value(i, col_label)


In [33]:
orient_path = Path(input_dir, 'Mouse-210_Orientation.tif')
write_image(orientation_array, orient_path)